### Solace Relativity v1.0 - Build Relativity and EJ Weighting

Read exploits db, use derivedValueIndicators and build data-values files and relativity scores, build expert-judgement template files, and build expert-judgement weighting files.

In [199]:
import pandas as pd
import json
import numpy as np
import math
import os


NUMBEROFEJSUBMITTERS = 3

#WIP

## Load Exploits, Protocols(tbd), and mappingTables (transactional and temporal) 
# TODO - next version, find each predictor columns and create mapping table entries
temporalDb=pd.read_excel('../temporalDB/TemporalDataset.xlsx')
exploitsDf = pd.read_csv('../exploits/historical_exploits.csv')
f = open('../governance/reference/mappingTablesSolace.json')
lookupTables = json.loads(f.read())
f.close()

# Init dfs for lookup tables
tvlSolaceLookupTable = pd.DataFrame(lookupTables['tvlSolaceLookup'])
auditsSolaceLookupTable = pd.DataFrame(lookupTables['auditSolaceLookup'])
oraclesSolaceLookupTable = lookupTables['oracleSolaceLookup'] # just values, not mappings: yes/no/unknown
categorySolaceLookupTable=pd.DataFrame(lookupTables['categorySolaceLookup'])


# TODO - category mapping ??

# Calc Exploit Damage ratio
exploitsDf['tvlPreExploit'] = exploitsDf['tvlPreExploit'] / 10 # confirm wtih Tim
exploitsDf['exploitDamage'] = exploitsDf['loss'] / exploitsDf['tvlPreExploit']
#exploitsDf['exploitDamage']= exploitsDf['exploitDamage'].astype(object).replace(np.nan, 'null')

# Calc time to exploit
exploitsDf[['launchDate','exploitDate']] = exploitsDf[['launchDate','exploitDate']].apply(pd.to_datetime) #if conversion required
exploitsDf['timeToExploit'] = (exploitsDf['exploitDate'] - exploitsDf['launchDate']).dt.days
exploitsDf['timeToExploit'] = exploitsDf['timeToExploit'].astype(object).replace(np.nan, 'null')

#temporalDb['categorySolace2']=temporalDb['tags'].str.get(0)


def lookup_categorySolace(category):
    if pd.isna(category):
        return 'Unknown'
    elif category not in list(categorySolaceLookupTable['category']):
        return 'other'
    else:
        return category

# Derive tvlSolace column
def lookup_tvlSolace(tvl):
    if math.isnan(tvl):
        return 'null'
    match = (tvlSolaceLookupTable['lowerLimit'] <= tvl) & (tvlSolaceLookupTable['upperLimit'] > tvl)
    tvlOut = tvlSolaceLookupTable['tvlSolace'][match]
    return tvlOut.values[0]


# Derive auditSolace column
def lookup_auditSolace(audits):
    if math.isnan(audits):
        return 'Unknown'
    match = (auditsSolaceLookupTable['auditsOg'] == audits)
    auditsOut = auditsSolaceLookupTable['auditSolace'][match]
    return auditsOut.values[0]


# Derive oracleSolace column
def lookup_oracleSolace(oracles,catalog):
    if pd.isna(catalog):
        return 'Unknown'
    elif pd.isna(oracles):
        return 'No'
    else:
        return "Yes"

# Reminder-add categorySolace to exploits too

#Solace judgement
#temporalDb['tags']=temporalDb['tags'].str.get(0)
#temporalDb['categorySolace3'] = temporalDb['tags'].apply(lookup_categorySolace)
exploitsDf['tvlSolace'] = exploitsDf['tvlPreExploit'].apply(lookup_tvlSolace)
temporalDb['tvlSolace'] = temporalDb['tvl'].apply(lookup_tvlSolace)
exploitsDf['auditSolace'] = exploitsDf['audits'].apply(lookup_auditSolace)
temporalDb['auditSolace'] = temporalDb['audits'].apply(lookup_auditSolace)            
exploitsDf['oracleSolace'] = exploitsDf.apply(lambda x: lookup_oracleSolace(x.oracles, x.category), axis=1)
temporalDb['oracleSolace'] = temporalDb.apply(lambda x: lookup_oracleSolace(x.oracles, x.tags), axis=1)

# Create columns for use in one way calcs
exploitsDf['tvlAvgDamage'] = exploitsDf.groupby(['tvlSolace'])['exploitDamage'].transform('mean')
exploitsDf['auditAvgDamage'] = exploitsDf.groupby(['auditSolace'])['exploitDamage'].transform('mean')
exploitsDf['oracleAvgDamage'] = exploitsDf.groupby(['oracleSolace'])['exploitDamage'].transform('mean')

averageDamage = exploitsDf['exploitDamage'].mean()

exploitsDf['tvlDamageRatio'] = exploitsDf['tvlAvgDamage'] / averageDamage
exploitsDf['auditDamageRatio'] = exploitsDf['auditAvgDamage'] / averageDamage
exploitsDf['oracleDamageRatio'] = exploitsDf['oracleAvgDamage'] / averageDamage

print('Average exploit damage ratio: ',averageDamage)

# Create one way 'tables', separate for readability 
# TVL
tvlOneWay = pd.DataFrame()
tvlOneWay['damageRatio'] = exploitsDf['tvlDamageRatio'].unique()
tvlOneWay['tvlSolace'] =exploitsDf['tvlSolace'].unique()
tvlOneWay['avgDamage'] = exploitsDf['tvlAvgDamage'].unique()

# AUDITS
auditOneWay = pd.DataFrame()
auditOneWay['damageRatio'] = exploitsDf['auditDamageRatio'].unique()
auditOneWay['auditSolace'] = exploitsDf['auditSolace'].unique()
auditOneWay['avgDamage'] = exploitsDf['auditAvgDamage'].unique()

# ORACLES
oracleOneWay = pd.DataFrame()
oracleOneWay['damageRatio'] = exploitsDf['oracleDamageRatio'].unique()
oracleOneWay['oracleSolace'] =exploitsDf['oracleSolace'].unique()
oracleOneWay['avgDamage'] = exploitsDf['oracleAvgDamage'].unique()

def write_json(df, fileName, drop):
    if drop == True:
        df.drop(['avgDamage'], axis=1, inplace=True)
    data = df.to_json(orient='columns')
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    with open(fileName, 'w') as outfile:
        outfile.write(data)

write_json(pd.merge(tvlOneWay, tvlSolaceLookupTable, how="right", on='tvlSolace'), '../governance/data-values/tvlDataValues.json', True)
write_json(pd.merge(auditOneWay, auditsSolaceLookupTable, how="right", on='auditSolace'), '../governance/data-values/auditDataValues.json', True)
write_json(oracleOneWay, '../governance/data-values/oracleDataValues.json', True)

print("Data values json files created in governance directory")

# Initialize EJ directories, files and weightings
def initialize_ejs(df):
    for i in df['damageRatio']:
        df['damageRatio'] = 1
    return df

# EJ weighting files
def create_weights(predictor):
    weightingsTemplate = '{"' + predictor + 'Basis": {"1": "data","2":"Ej1"},"weight": {"1": 0.75, "2":0.25}}'
    weightingsTemplateJson = json.loads(weightingsTemplate)
    weightingsTemplateDf = pd.DataFrame(weightingsTemplateJson)
    write_json(weightingsTemplateDf, '../governance/weightings/'+predictor+'Weighting.json', False)
create_weights('audit')
create_weights("oracle")
create_weights("tvl")
create_weights("category")

for i in range(NUMBEROFEJSUBMITTERS):
    print("Initializing EJ files " + str(i+1) + "...")
    write_json(initialize_ejs(tvlOneWay), '../governance/expert-judgement/' + str(i+1) + '/ejTvlDataValues.json', False)
    write_json(initialize_ejs(oracleOneWay), '../governance/expert-judgement/' + str(i+1) + '/ejOracleDataValues.json', False)
    write_json(initialize_ejs(auditOneWay), '../governance/expert-judgement/' + str(i+1) + '/ejAuditDataValues.json', False)
    print("EJ Submitter" + str(i+1) + " created in governance directory")
    # Create weighting files per predictor with basis and each expert   

print("\nRelativities have been initialed to 1. Update files in governance directory to incorporate expert judgement into realativity scoring.")




Average exploit damage ratio:  0.24146191913870418
Data values json files created in governance directory
Initializing EJ files 1...
EJ Submitter1 created in governance directory
Initializing EJ files 2...
EJ Submitter2 created in governance directory
Initializing EJ files 3...
EJ Submitter3 created in governance directory

Relativities have been initialed to 1. Update files in governance directory to incorporate expert judgement into realativity scoring.


### Temporal Data relativities

In [200]:
# Data values
def open_data(file):
    Data = open(file)
    lookupTables = json.loads(Data.read())
    Data.close()
    return lookupTables

def add_column(file,key,database,merger):
    lookup=open_data(file)
    attributeSolaceRelativity=lookup[key]
    attributeFrame=pd.DataFrame(attributeSolaceRelativity)
    combinedAuditTable1=pd.merge(database,attributeFrame,left_on=merger, right_on=merger, how='left')
    return combinedAuditTable1

def calc_finalRelativity(weightLookup,combinedtable2,column1,column2,key,key_weight):
    lookupTable=open_data(weightLookup)
    #lookupTable=open_data('../governance/weightings/auditWeighting.json')
    lookupFrame=pd.DataFrame(lookupTable)
    data_weight=lookupFrame[lookupFrame[key_weight]=='data']['weight']
    solaceRM_weight=lookupFrame[lookupFrame[key_weight]=='Ej1']['weight']
    combinedtable2[key]=(column1*data_weight.values)+(column2*solaceRM_weight.values) #+(column1*solaceRM_weight.values)+(column2*solaceRM_weight.values)
    return combinedtable2

def aggregateTables(weightLookup,file,file2,key,key1,key2,database,merger,key_weight):
    # Add audit Relativities
    column_test=add_column(file,key,database,merger)
    column_test2=add_column(file2,key,column_test,merger)
    column_test3=calc_finalRelativity(weightLookup,column_test2,column_test2[key1],column_test2[key2],key,key_weight)
    return column_test3

AuditRelativityColumns=aggregateTables('../governance/weightings/auditWeighting.json','../governance/data-values/auditRelativities.json','../governance/expert-judgement/solaceRM/ejAuditDataValues.json','auditSolaceRelativities','relativityAuditData','relativityAuditEj1',temporalDb,'auditSolace','auditBasis')
TVLRelativityColumns=aggregateTables('../governance/weightings/tvlWeighting.json','../governance/data-values/tvlRelativities.json','../governance/expert-judgement/solaceRM/ejTvlDataValues.json','tvlSolaceRelativities','relativityTvlData','relativityTvlEj1',AuditRelativityColumns,'tvlSolace','tvlBasis')
OracleRelativityColumns=aggregateTables('../governance/weightings/oracleWeighting.json','../governance/data-values/oracleRelativities.json','../governance/expert-judgement/solaceRM/ejOracleDataValues.json','oracleSolaceRelativities','relativityOracleData','relativityOracleEj1',TVLRelativityColumns,'oracleSolace','oracleBasis')
CategoryRelativityColumns=aggregateTables('../governance/weightings/categoryWeighting.json','../governance/data-values/categoryRelativities.json','../governance/expert-judgement/solaceRM/ejCategoryDataValues.json','categorySolaceRelativities','relativityCategoryData','relativityCategoryEj1',OracleRelativityColumns,'categorySolace','categoryBasis')

#OracleRelativityColumns.iloc[:,[17,18,19,20,21,22,23,24,25]]
CategoryRelativityColumns['finalRelativity']=CategoryRelativityColumns['auditSolaceRelativities']*CategoryRelativityColumns['tvlSolaceRelativities']*CategoryRelativityColumns['oracleSolaceRelativities']*CategoryRelativityColumns['categorySolaceRelativities']
CategoryRelativityColumns


,name,address,symbol,audits,chains,oracles,tvl,forkedFrom,SolaceID,appId,...,relativityTvlData,relativityTvlEj1,tvlSolaceRelativities,relativityOracleData,relativityOracleEj1,oracleSolaceRelativities,relativityCategoryData,relativityCategoryEj1,categorySolaceRelativities,finalRelativity
0,Curve,0xd533a949740bb3306d119cc777fa900ba034cd52,CRV,2.0,"['Ethereum', 'Avalanche', 'Fantom', 'Polygon',...",['Chainlink'],1.916566e+10,NaN,curve,curve,...,1,1,1.0,1,2,1.25,1.0,2.0,1.250,1.562500
1,Convex Finance,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,CVX,2.0,['Ethereum'],NaN,1.337217e+10,NaN,convex,convex,...,1,1,1.0,1,1,1.00,1.0,2.0,1.250,1.250000
2,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],1.284725e+10,NaN,aave,aave-amm,...,1,1,1.0,1,2,1.25,1.0,1.0,1.000,1.250000
3,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],1.284725e+10,NaN,aave,aave-safety-module,...,1,1,1.0,1,2,1.25,1.0,2.0,1.250,1.562500
4,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],1.284725e+10,NaN,aave,aave,...,1,1,1.0,1,2,1.25,1.0,1.0,1.000,1.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,ARCx,eth:0x1321f1f1aa541a56c31682c57b80ecfccd9bb288,ARCx,0.0,['Ethereum'],NaN,0.000000e+00,NaN,arcx,arcx,...,1,3,1.5,1,1,1.00,1.0,2.0,1.250,2.343750
131,The Sandbox,0x3845badade8e6dff049820680d1f14bd3903a5d0,SAND,0.0,"['Ethereum', 'Polygon']",NaN,0.000000e+00,NaN,the,the-graph,...,1,3,1.5,1,1,1.00,1.0,2.0,1.250,2.343750
132,LooksRare,0xf4d2888d29d722226fafa5d9b24f9164c092421e,LOOKS,0.0,['Ethereum'],NaN,0.000000e+00,NaN,looksrare,looksrare,...,1,3,1.5,1,1,1.00,1.0,1.5,1.125,2.109375
133,The Open DAO SOS,0x3b484b82567a09e2588a13d54d032153f0c0aee0,SOS,2.0,['Ethereum'],NaN,0.000000e+00,NaN,the,the-graph,...,1,3,1.5,1,1,1.00,1.0,2.0,1.250,1.875000
